# Installing libraries

In [ ]:
!pip install -r https://raw.githubusercontent.com/McGill-MMA-EnterpriseAnalytics/SafeRide_Dtection/refs/heads/main/requirements.txt

Ignoring colorama: markers 'python_version >= "3.11" and python_version < "4.0" and platform_system == "Windows"' don't match your environment
Ignoring pywin32: markers 'python_version >= "3.11" and python_version < "4.0" and sys_platform == "win32"' don't match your environment
Ignoring waitress: markers 'python_version >= "3.11" and python_version < "4.0" and platform_system == "Windows"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

# Adding Kaggle Authkey

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yashsethi24","key":"edbf0036ebd673985f86ae4176a42b53"}'}

In [2]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Downloading dataset for training

In [3]:
!kaggle datasets download -d andrewmvd/helmet-detection
!unzip helmet-detection.zip -d helmet_data

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/helmet-detection
License(s): CC0-1.0
Archive:  helmet-detection.zip
  inflating: helmet_data/annotations/BikesHelmets0.xml  
  inflating: helmet_data/annotations/BikesHelmets1.xml  
  inflating: helmet_data/annotations/BikesHelmets10.xml  
  inflating: helmet_data/annotations/BikesHelmets100.xml  
  inflating: helmet_data/annotations/BikesHelmets101.xml  
  inflating: helmet_data/annotations/BikesHelmets102.xml  
  inflating: helmet_data/annotations/BikesHelmets103.xml  
  inflating: helmet_data/annotations/BikesHelmets104.xml  
  inflating: helmet_data/annotations/BikesHelmets105.xml  
  inflating: helmet_data/annotations/BikesHelmets106.xml  
  inflating: helmet_data/annotations/BikesHelmets107.xml  
  inflating: helmet_data/annotations/BikesHelmets108.xml  
  inflating: helmet_data/annotations/BikesHelmets109.xml  
  inflating: helmet_data/annotations/BikesHelmets11.xml  
  inflating: helmet_data/annotations/BikesHelmets110.xml 

# Dividing dataset for training and test set

In [6]:
import os
import random
import shutil
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

# Paths
base_path = '/content/helmet_data'
images_path = os.path.join(base_path, 'images')
annotations_path = os.path.join(base_path, 'annotations')
output_path = '/content/helmet_data_split'  # New base folder

# Create output folders
for split in ['train', 'val']:
    os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_path, split, 'labels'), exist_ok=True)

# Step 1: Find all images that have corresponding annotations
image_files = [f for f in os.listdir(images_path) if f.endswith('.png')]
annotation_files = [f.replace('.xml', '') for f in os.listdir(annotations_path) if f.endswith('.xml')]

# Keep only images that have annotations
valid_images = [f for f in image_files if f.replace('.png', '') in annotation_files]

print(f"Total valid images with annotations: {len(valid_images)}")

# Step 2: Split into train and val
train_imgs, val_imgs = train_test_split(valid_images, test_size=0.2, random_state=42)  # 80% train, 20% val
splits = {'train': train_imgs, 'val': val_imgs}

# Helper: XML to YOLO TXT converter
def convert_annotation(xml_file, txt_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    with open(txt_file, 'w') as f:
        for obj in root.findall('object'):
            cls = obj.find('name').text
            # Assuming 'plate' is class 0, adjust if multiple classes later
            cls_id = 0
            xmlbox = obj.find('bndbox')
            xmin = int(xmlbox.find('xmin').text)
            xmax = int(xmlbox.find('xmax').text)
            ymin = int(xmlbox.find('ymin').text)
            ymax = int(xmlbox.find('ymax').text)

            # Convert to YOLO format
            x_center = (xmin + xmax) / 2 / width
            y_center = (ymin + ymax) / 2 / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height

            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

# Step 3: Move and convert
for split_name, img_list in splits.items():
    for img_file in img_list:
        img_src = os.path.join(images_path, img_file)
        img_dst = os.path.join(output_path, split_name, 'images', img_file)
        shutil.copy(img_src, img_dst)

        # Convert annotation
        xml_file = os.path.join(annotations_path, img_file.replace('.png', '.xml'))
        txt_file = os.path.join(output_path, split_name, 'labels', img_file.replace('.png', '.txt'))
        convert_annotation(xml_file, txt_file)

print("✅ Full dataset with train/val split prepared successfully!")


Total valid images with annotations: 764
✅ Full dataset with train/val split prepared successfully!


# Adding Yaml file for YOLO

In [ ]:
dataset_yaml = """
path: ../helmet_data_split
train: train/images
val: train/images

names:
  0: Helmet
"""

with open('/content/helmet_data_split/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

print("✅ dataset.yaml updated correctly with val field!")


✅ dataset.yaml updated correctly with val field!


# Training model with more epochs + Early Stopping + more training images

In [10]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [14]:
from ultralytics import YOLO
import os

# --------------- Settings ----------------
model_type = 'yolov8m.pt'
imgsz = 800
epochs = 60                     # More than 20 for longer training
patience = 20                    # Early stopping patience
batch_size = 16
data_path = '/content/helmet_data_split/dataset.yaml'
save_dir = '/content/yolo_helmet_final'
run_name = f"{model_type.replace('.pt','')}_imgsz{imgsz}_ep{epochs}_final"

# --------------- Training ----------------
model = YOLO(model_type)

results = model.train(
    data=data_path,
    epochs=epochs,
    imgsz=imgsz,
    save=True,
    project=save_dir,
    name=run_name,
    batch=batch_size,
    device=0,
    workers=4,
    optimizer='SGD',
    lr0=0.001,
    lrf=0.1,
    weight_decay=0.0005,
    momentum=0.937,
    patience=patience,
    cache=True,
    save_period=10,
    exist_ok=True,

    # Augmentations
    augment=True,
    conf=0.5,
    iou=0.5,
    mosaic=0.3,
    mixup=0.1,
    hsv_h=0.01,
    hsv_s=0.3,
    hsv_v=0.2,
    flipud=0.0,
    fliplr=0.3,
    degrees=3.0,
    translate=0.05,
    scale=0.1,
    shear=0.05,
    perspective=0.0,
    amp=True,
    agnostic_nms=False,
)

print("✅ Final training completed!")


Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/helmet_data_split/dataset.yaml, epochs=60, time=None, patience=20, batch=16, imgsz=800, save=True, save_period=10, cache=True, device=0, workers=4, project=/content/yolo_helmet_final, name=yolov8m_imgsz800_ep60_final, exist_ok=True, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=0.5, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

train: Scanning /content/helmet_data_split/train/labels.cache... 611 images, 3 backgrounds, 12 corrupt: 100%|██████████| 611/611 [00:00<?, ?it/s]

train: /content/helmet_data_split/train/images/BikesHelmets103.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     408.72      81.502      71.037      83.004]
train: /content/helmet_data_split/train/images/BikesHelmets279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      194.5          56         103         100]
train: /content/helmet_data_split/train/images/BikesHelmets326.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        157        56.5          86          97]
train: /content/helmet_data_split/train/images/BikesHelmets441.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      310.5        41.5          61          63]
train: /content/helmet_data_split/train/images/BikesHelmets444.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [        299          48          78          80]
train: /content/helmet_data_split/train/

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.7GB RAM): 100%|██████████| 599/599 [00:08<00:00, 71.53it/s]

...

      60/60      10.1G     0.5409     0.2604     0.8983         16        800: 100%|██████████| 38/38 [00:31<00:00,  1.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.06it/s]

                   all        599       1135      0.966      0.984       0.99      0.904



60 epochs completed in 0.772 hours.
Optimizer stripped from /content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/last.pt, 52.0MB
Optimizer stripped from /content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/best.pt, 52.0MB

Validating /content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/best.pt...
Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:19<00:00,  1.01s/it]


                   all        599       1135      0.961      0.975       0.99      0.864
Speed: 0.2ms preprocess, 26.7ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/yolo_helmet_final/yolov8m_imgsz800_ep60_final
✅ Final training completed!


In [16]:
import shutil
import os

# ----------- Define paths ------------
save_dir = '/content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights'
destination_dir = '/content/saved_helmet_model'

# Create destination directory
os.makedirs(destination_dir, exist_ok=True)

# ----------- Save best.pt and last.pt ------------
shutil.copy(os.path.join(save_dir, 'best.pt'), os.path.join(destination_dir, 'best.pt'))
shutil.copy(os.path.join(save_dir, 'last.pt'), os.path.join(destination_dir, 'last.pt'))

print("✅ Helmet model weights saved to /content/saved_helmet_model/")

# ----------- Optional: Save training plots ------------
plots_dir = '/content/helmet_data_split/yolov8m_imgsz800_ep100_final/'
plot_files = ['results.png', 'results.csv']  # Standard YOLO outputs

for plot_file in plot_files:
    plot_path = os.path.join(plots_dir, plot_file)
    if os.path.exists(plot_path):
        shutil.copy(plot_path, os.path.join(destination_dir, plot_file))

print("✅ Training plots saved (if available).")


✅ Helmet model weights saved to /content/saved_helmet_model/
✅ Training plots saved (if available).


In [19]:
# Zip it
!zip -r /content/saved_model.zip /content/yolo_helmet_final/yolov8m_imgsz800_ep60_final

# Now manually download /content/saved_model.zip from the sidebar!

  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/ (stored 0%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/F1_curve.png (deflated 19%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/results.csv (deflated 61%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/train_batch0.jpg (deflated 4%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/val_batch1_pred.jpg (deflated 6%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/ (stored 0%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/epoch50.pt (deflated 8%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/last.pt (deflated 8%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/epoch0.pt (deflated 7%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/best.pt (deflated 8%)
  adding: content/yolo_helmet_final/yolov8m_imgsz800_ep60_final/weights/epoch10.p